<a href="https://colab.research.google.com/github/andyly13/clause-extraction/blob/main/clause_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

In [ ]:
import sparknlp
spark = sparknlp.start()

print("Spark NLP version: {}".format(sparknlp.version()))
print("Apache Spark version: {}".format(spark.version))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pikepdf
from pikepdf import Pdf

In [ ]:
# Andrew Ly Clause Extraction Code (02/07/2024)
# getting all of the longest files in each folder in sorted

!pip install PyPDF2

from multiprocessing import Pool
import PyPDF2
import os

# ----- functions -------

# returns a generator that holds all of the ID folders
def contract_generator(root_directory):
    contract_folders = os.listdir(root_directory)
    for id in contract_folders:
        contracts = os.listdir(os.path.join(root_directory, id))
        yield contracts, id

# appends the longest contract to the pdfs_to_check list to be used for text-conversion?
def add_pdfs(root_directory, longest_contract_dict):
    contract_gen = contract_generator(root_directory)
    while True:
      try:
        contracts, id = next(contract_gen)

        if len(contracts) == 1:
          longest_contract_dict[int(id)] = contracts[0] # add first and only pdf
          print("Contract " + contracts[0] + " added from file " + id)
        elif len(contracts) == 0: # handling empty folder, think there was only one but it broke everything LOL
          longest_contract_dict[int(id)] = 'None'
          print("File " + id + " was empty. Added 'None' to dict.")
        else:
          longest_contract = get_longest_contract(root_directory, contracts, id)
          longest_contract_dict[int(id)] = longest_contract
          print("Contract " + longest_contract + " added from file " + id)
      except StopIteration: # occurs when generator gets to the end of the file
        break

# takes a list of contracts, returns the one with the most pages
def get_longest_contract(root_directory, contracts, id):
  max_pages = 0
  longest_contract = contracts[0]

  with Pdf.open(os.path.join(root_directory, id, contracts[0])) as longest_pdf:
    max_pages = len(longest_pdf.pages)

  for i in range(1, len(contracts)):
    with Pdf.open(os.path.join(root_directory, id, contracts[i])) as temp_pdf:
      temp_pages = len(temp_pdf.pages)

      if temp_pages > max_pages:
        longest_contract = contracts[i]
        max_pages = temp_pages

  # with open(os.path.join(root_directory, id, contracts[0]), 'rb') as fileObj:
  #   pdfReader = PyPDF2.PdfReader(fileObj)
  #   if pdfReader.is_encrypted:
  #     return 'None (Encrypted)' # skipping for now
  #   else:
  #     max_pages = len(pdfReader.pages)

  # # gets longest contract
  # for contract in contracts:
  #   with open(os.path.join(root_directory, id, contract), 'rb') as fileObj:
  #     pdfReader = PyPDF2.PdfReader(fileObj)
  #     if len(pdfReader.pages) > max_pages:
  #       longest_contract = contract
  #       max_pages= len(pdfReader.pages)

  return longest_contract


# ----- calls --------

longest_contract_dict = {} # folder ID is the key and contract path is the value
root_directory = '/content/drive/Shareddrives/Contract Data/Contracts (PDF)/1 Sorted'

with Pool() as pool: # not sure if this does anything, but should help it run faster using multiple CPU cores
  add_pdfs(root_directory, longest_contract_dict)




In [ ]:
# Andrew Ly - importing dictionary to CSV
# Longest contract spreadsheet is viewable in google sheets

import csv

def export_to_csv(data, file_name):
    with open(file_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(['ID', 'Longest Contract'])  # Write header
        for id, contract in data.items():
            writer.writerow([id, contract])

export_to_csv(longest_contract_dict, 'contracts.csv')


In [ ]:
# Andrew Ly - Copying longest pdfs and moving to separate directory

# DO NOT RERUN, WILL ADD DUPLICATE FILES!!!!!
# DO NOT RERUN, WILL ADD DUPLICATE FILES!!!!!
# DO NOT RERUN, WILL ADD DUPLICATE FILES!!!!!

!pip install gspread google-auth

import gspread
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.auth import default
import os
import shutil

# Authenticate and create a client
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)


#---------functions------------#

# Iterate through all rows except first (these are the column titles)
# and then create a copy and move to Longest Contracts folder within Contracts (PDF)
def move_file_copies(sheet, root_directory):
  original_folder = '1 Sorted'
  new_folder = 'Longest Contracts'

  for i in range(569, spreadsheet.row_count):
    row = spreadsheet.row_values(i)
    print("At row " + str(row))
    folder_id = row[0]
    pdf_name = row[1]
    path_to_pdf = os.path.join(root_directory, original_folder, folder_id, pdf_name)

    if os.path.isfile(path_to_pdf):
      # creating folder with ID name in 'Longest Contracts' folder
      new_folder_path = os.path.join(root_directory, new_folder, folder_id)
      os.makedirs(new_folder_path, exist_ok=True)

      # making copy and putting it into new folder
      shutil.copy(path_to_pdf, new_folder_path)




#---------function calls------------#
spreadsheet = gc.open("LongestContracts").sheet1
root_directory = '/content/drive/Shareddrives/Contract Data/Contracts (PDF)'

print("moving files... ")
move_file_copies(spreadsheet, root_directory)
print("files have been copied and moved to Longest Contracts")





moving files... 
At row ['126898', 'Deal_126898_Sublicense2.pdf']
At row ['126977', 'Deal_126977_Equity2.pdf']
At row ['127130', 'Deal_127130_Supply1.pdf']
At row ['127155', 'Deal_127155_Co-development_License2.pdf']
At row ['127356', 'Deal_127356_Development_License6.pdf']
At row ['127357', 'Deal_127357_Amendment1.pdf']
At row ['127427', 'Deal_127427_Security1.pdf']
At row ['127445', 'Deal_127445_Co-Promotion1.pdf']
At row ['127453', 'Deal_127453_Research_License3.pdf']
At row ['127498', 'Deal_127498_Research_Co-Promotion_License_Collaboration1.pdf']
At row ['127563', 'Deal_127563_Asset Purchase1.pdf']
At row ['127586', 'Deal_127586_Amendment5.pdf']
At row ['127615', 'Deal_127615_Development_License2.pdf']
At row ['127659', 'Deal_127659_Supply_Amendment4.pdf']
At row ['127715', 'Deal_127715_Amendment3.pdf']
At row ['127729', 'Deal_127729_Asset Purchase1.pdf']
At row ['127743', 'Deal_127743_Amendment1.pdf']
At row ['127750', 'Deal_127750_Amendment4.pdf']
At row ['127787', 'Deal_127787_

IndexError: list index out of range